In [ ]:
"""

Author: Camila Losada
"""

In [1]:
import numpy as np
from joblib import Parallel, delayed
from tqdm import tqdm
from pathlib import Path
from typing import Dict, List
from compdec_v4pfclip_01.utils.structures.neuron_data import NeuronData
from compdec_v4pfclip_01.utils.structures.population_data import PopulationData
import pandas as pd
import glob

In [2]:
# Define parameters
params = [
    {
        "loc": "contra",
        "event": "sample_on",
        "time_before": 500,
        "time_after": 1000,
        "select_block": 1,
        "error_type": 0,
    },
    {
        "loc": "contra",
        "event": "test_on_1",
        "time_before": 500,
        "time_after": 600,
        "select_block": 1,
        "error_type": 0,
    },
    {
        "loc": "ipsi",
        "event": "sample_on",
        "time_before": 500,
        "time_after": 1000,
        "select_block": 1,
        "error_type": 0,
    },
    {
        "loc": "ipsi",
        "event": "test_on_1",
        "time_before": 500,
        "time_after": 600,
        "select_block": 1,
        "error_type": 0,
    }
]

In [11]:
def read_and_compute(path,params):
    neu=NeuronData.from_python_hdf5(path)
    neu=neu.get_align_on(params=params,delete_att=['sp_samples'],inplace=True)
    return neu

In [ ]:
# paths
filepaths = {
    "lip": "../session_struct/lip/neurons/",
    "pfc": "",
    "v4": "",
}
path_list={}
for area in filepaths.keys():
    path = filepaths[area]
    neu_path = path + "*neu.h5"
    path_list[area] = glob.glob(neu_path)

In [13]:
population = Parallel(n_jobs=-1)(
    delayed(read_and_compute)(path,params) for path in tqdm(path_list['lip'][:10])
)

100%|██████████| 10/10 [00:00<00:00, 2094.95it/s]
Traceback (most recent call last):
  File "/home/INT/losada.c/miniforge3/envs/compdec/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 422, in _on_run
    cmd.send(self.sock)
  File "/home/INT/losada.c/miniforge3/envs/compdec/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_net_command.py", line 111, in send
    sock.sendall(("Content-Length: %s\r\n\r\n" % len(as_bytes)).encode("ascii"))
OSError: [Errno 9] Bad file descriptor

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/INT/losada.c/miniforge3/envs/compdec/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_daemon_thread.py", line 53, in run
    self._on_run()
  File "/home/INT/losada.c/miniforge3/envs/compdec/lib/python3.12/site-packages/debugpy/_vendored/pydevd/_pydevd_bundle/pydevd_comm.py", line 432, in _on_run
    sel

In [14]:
len(population[0])

TypeError: object of type 'NeuronData' has no len()

In [15]:
popu = PopulationData(population)
popu.to_python_hdf5('population.h5')
popu = PopulationData.from_python_hdf5('population.h5')

In [16]:
popu.__dict__.keys()

dict_keys(['population', 'comment', '_PopulationData__created'])

In [17]:
popu.population